In [6]:
!pip3 install scikit-learn
!pip3 install torch


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
import numpy as np

c:\Source\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# only for running locally
dfs = []
for filename in os.listdir('features_ring'):
    if filename[-4:] == '.tsv':
        dfs.append(pd.read_csv('features_ring/' + filename, sep='\t'))
df = pd.concat(dfs)
df

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'features_ring'

In [4]:
# don't need to execute - only to export df for training in cloud
df.to_csv('contact_df.csv')

NameError: name 'df' is not defined

In [9]:
!pip3 install google-colab

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached urllib3

  ERROR: Command errored out with exit status 1:
   command: 'c:\program files\python39\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\marco\\AppData\\Local\\Temp\\pip-install-ve27fccd\\pandas_11b98f5c8eae4eb0af6c7248a9bc3adf\\setup.py'"'"'; __file__='"'"'C:\\Users\\marco\\AppData\\Local\\Temp\\pip-install-ve27fccd\\pandas_11b98f5c8eae4eb0af6c7248a9bc3adf\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\marco\AppData\Local\Temp\pip-wheel-a3v50nwz'
       cwd: C:\Users\marco\AppData\Local\Temp\pip-install-ve27fccd\pandas_11b98f5c8eae4eb0af6c7248a9bc3adf\
  Complete output (1000 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-3.9\pandas
  copy

In [3]:
# only for running on drive for training
#from google.colab import drive
#drive.mount('/content/drive')
df = pd.read_csv('contact_df.csv')

In [4]:
df.dropna(inplace=True)

# Define ground truth values
y = df['Interaction'].astype('category')
y_oneHot = pd.get_dummies(y)
y

0               VDW
1             HBOND
2             HBOND
3         PIPISTACK
4             HBOND
            ...    
454188          VDW
454189        HBOND
454190          VDW
454191        HBOND
454192        HBOND
Name: Interaction, Length: 454193, dtype: category
Categories (6, object): ['HBOND', 'IONIC', 'PICATION', 'PIPISTACK', 'SSBOND', 'VDW']

In [6]:
# Define training features
X = df[['s_rsa', 's_up', 's_down', 's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 
        't_rsa', 't_up', 't_down', 't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5']]

X = X.fillna({'s_up': X.s_up.mode()[0], 's_down': X.s_down.mode()[0],
                's_phi': X.s_phi.mode()[0], 's_psi': X.s_psi.mode()[0], 's_a1': X.s_a1.mode()[0],
                's_a2': X.s_a2.mode()[0], 's_a3': X.s_a3.mode()[0], 's_a4': X.s_a4.mode()[0],
                's_a5': X.s_a5.mode()[0], 't_up': X.t_up.mode()[0],
                't_down': X.t_down.mode()[0], 't_phi': X.t_phi.mode()[0], 't_psi': X.t_psi.mode()[0],
                't_a1': X.t_a1.mode()[0], 't_a2': X.t_a2.mode()[0], 't_a3': X.t_a3.mode()[0],
                't_a4': X.t_a4.mode()[0], 't_a5': X.t_a5.mode()[0]})

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled

,s_rsa,s_up,s_down,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_rsa,t_up,t_down,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5
0,-0.612926,1.212548,1.272426,-0.080637,1.435561,1.032979,0.438314,-1.675155,-0.535573,-1.967293,-0.926898,0.482516,1.366067,-1.502723,1.362867,-0.045132,0.558611,1.029760,0.741296,0.813673
1,0.235709,0.077481,-0.558473,0.268180,-0.783304,1.337499,-1.430106,0.702567,-0.124882,-0.467409,-0.118231,-0.267989,-0.712127,0.257241,-0.791289,1.476577,0.135358,0.690168,0.232221,1.818554
2,1.797971,-1.341352,-0.009203,0.266785,-0.772920,0.928827,0.998308,0.620114,-0.436212,0.636456,1.398591,-1.318697,0.500153,0.283161,-0.812362,1.003587,0.531949,-1.773429,-0.528128,-2.075994
3,-0.096994,1.070665,-0.741563,-1.206618,1.219451,-1.006413,-0.511333,0.894341,-0.687926,0.311533,-0.926898,0.932819,1.192884,-1.816354,1.560677,-0.989173,-0.458974,0.875965,-0.678240,0.242084
4,-0.096994,1.070665,-0.741563,-1.206618,1.219451,-1.006413,-0.511333,0.894341,-0.687926,0.311533,-0.926898,0.932819,1.192884,-1.816354,1.560677,-0.989173,-0.458974,0.875965,-0.678240,0.242084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454188,1.161494,-2.050769,0.173887,-0.373643,-0.294621,-0.389438,1.875560,0.634473,0.904054,1.341807,0.854911,-1.618899,1.019702,0.564392,-0.796048,-1.001773,-0.900002,-0.746055,1.130717,-0.597855
454189,-0.786511,0.503131,-0.558473,0.021218,1.453084,-1.006413,-0.511333,0.894341,-0.687926,0.311533,1.901152,-2.219303,-0.538944,4.190592,-0.494914,-0.386305,2.031665,0.608016,0.890320,1.290104
454190,-0.878125,0.645015,-0.009203,-0.975003,1.163638,-1.237531,-0.465554,1.005515,0.184240,0.563489,-0.433474,1.082920,-1.404858,-1.406819,1.223517,0.237886,1.118505,1.451982,-1.157946,0.939918
454191,1.682248,-1.341352,-1.107742,-1.269405,-0.158335,1.807671,-0.480459,0.265285,-0.555445,1.082368,2.335182,-2.369404,-1.058493,3.708481,-0.372558,-0.386305,2.031665,0.608016,0.890320,1.290104


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_oneHot, test_size=0.1, random_state=0)
kf = KFold(n_splits=10, shuffle=True)

In [8]:
class ContactNet(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_layers_dim=[]):
        super().__init__()
        self.layers = nn.ModuleList()
        if len(hidden_layers_dim) == 0:
            self.layers = self.layers.append(nn.Linear(input_dim, num_classes))
        else:
            for layer_idx in range(len(hidden_layers_dim)):
                if layer_idx == 0:  # first layer, from input to hidden
                    self.layers = self.layers.append(nn.Linear(input_dim, hidden_layers_dim[layer_idx]))
                else:  # hidden layers, depending on the input
                    self.layers = self.layers.append(nn.Linear(hidden_layers_dim[layer_idx-1], hidden_layers_dim[layer_idx]))
            self.layers = self.layers.append(nn.Linear(hidden_layers_dim[-1], num_classes))  # final output layer
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=.1)
            if module.bias is not None:
                module.bias.data.zero_()
  
    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
        if x.dtype != torch.float32:
            x = x.float()
        if len(self.layers) == 1:
            return self.layers[0](x)
        else:
            for layer in self.layers[:-1]:
                x = F.relu(layer(x))
        return F.log_softmax(x, dim=1)#self.layers[-1](x)

input_size = X.shape[1]  # The number of input features
num_classes = y_oneHot.shape[1] # The number of output classes

model = ContactNet(input_size, num_classes, [256,256,256])

# Criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

summary(model)

Layer (type:depth-idx)                   Param #
ContactNet                               --
├─ModuleList: 1-1                        --
│    └─Linear: 2-1                       5,376
│    └─Linear: 2-2                       65,792
│    └─Linear: 2-3                       65,792
│    └─Linear: 2-4                       1,542
Total params: 138,502
Trainable params: 138,502
Non-trainable params: 0

In [14]:
# training
num_epochs = 20
for train_index, val_index in kf.split(X_train):
    X_train_fold = X_train.values[train_index]
    y_train_fold = y_train.values[train_index]
    X_val_fold = X_train.values[val_index]
    y_val_fold = y_train.values[val_index]

    for epoch in range(num_epochs):
        out = model(torch.tensor(X_train_fold))
        y_train_fold_labels = np.argmax(y_train_fold, axis=1)
        loss = criterion(out, torch.tensor(y_train_fold_labels).long())
        #loss = criterion(out, torch.tensor(y_train_fold))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print (f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

        val_outputs = model(X_val_fold)
        y_val_fold_labels = np.argmax(y_val_fold, axis=1)
        val_loss =  criterion(val_outputs, torch.tensor(y_val_fold_labels).long())
        _, val_preds = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_val_fold_labels, val_preds)
        
        print(f'Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}')

Epoch 1/20, Loss: 4.635767936706543
Validation Loss: 4.230266571044922, Validation Accuracy: 0.5027643231077842
Epoch 2/20, Loss: 4.232497692108154
Validation Loss: 3.8248305320739746, Validation Accuracy: 0.6220460883604872
Epoch 3/20, Loss: 3.827828884124756
Validation Loss: 3.422752618789673, Validation Accuracy: 0.6550222613630804
Epoch 4/20, Loss: 3.4266676902770996
Validation Loss: 3.031428813934326, Validation Accuracy: 0.6591809775429326
Epoch 5/20, Loss: 3.0363337993621826
Validation Loss: 2.661256790161133, Validation Accuracy: 0.659645775233622
Epoch 6/20, Loss: 2.6670634746551514
Validation Loss: 2.3243367671966553, Validation Accuracy: 0.659988257742551
Epoch 7/20, Loss: 2.3309762477874756
Validation Loss: 2.0319340229034424, Validation Accuracy: 0.659988257742551
Epoch 8/20, Loss: 2.039217233657837
Validation Loss: 1.790675163269043, Validation Accuracy: 0.6599393316698469
Epoch 9/20, Loss: 1.798357367515564
Validation Loss: 1.5997873544692993, Validation Accuracy: 0.6598

In [13]:
val_preds

tensor([241,   0,   0,  ..., 241,   0,  10])

In [12]:
y_val_fold

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0]], dtype=uint8)

In [40]:
class EarlyStopping: 
    def __init__(self, tolerance, min_delta): 
        self.tolerance = tolerance 
        self.min_delta = min_delta 
        self.counter = 0 
        self.early_stop = False 
    
    def __call__(self, train_loss, validation_loss): 
        if (validation_loss - train_loss) > self.min_delta: 
            self.counter +=1 
            if self.counter >= self.tolerance: 
                self.early_stop = True

early_stopping = EarlyStopping(tolerance=5, min_delta=0.01)

# training
num_epochs = 20
for train_index, val_index in kf.split(X_train):
    print(train_index)
    X_train_fold = X_train.values[train_index]
    y_train_fold = y_train.values[train_index]
    X_val_fold = X_train.values[val_index]
    y_val_fold = y_train.values[val_index]

    for epoch in range(num_epochs):
        out = model(torch.from_numpy(X_train_fold).float())
        loss = criterion(out, y_train_fold)



        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print (f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

        with torch.no_grad():  # Evaluation mode, no need to compute gradients
            val_outputs = model(X_val_fold)
            # val_loss = criterion(val_outputs, y_val_fold)
            val_loss = criterion(val_outputs, torch.from_numpy(y_val_fold).long())

            _, val_preds = torch.max(val_outputs, 1)
            val_accuracy = accuracy_score(y_val_fold.cpu(), val_preds.cpu())
       
            print(f'Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}')

            # Check for early stopping
            early_stopping(loss.item(), val_loss.item())
            if early_stopping.early_stop:
                print('Early stopping triggered...')
                break

    # If early stopping was triggered, break the outer loop as well
    if early_stopping.early_stop:
        break


[     0      1      2 ... 408770 408771 408772]


TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not numpy.ndarray